In [6]:
import numpy as np
import os
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from PIL import Image
import os,sys
from pathlib import Path

from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC, SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

from helpers import img_crop, extract_features_2d, value_to_class

## Loading the Original Training Set For Linear Modelling

In [7]:
image_path = str(Path.cwd().parent) + "/data/original/images/"
files = os.listdir(image_path)
n = len(files) 
print("Loading " + str(n) + " images")
imgs = [mpimg.imread(image_path + files[i]) for i in range(n)]
print(files[0])

Loading 100 images
satImage_052.png


In [8]:
gt_dir = str(Path.cwd().parent) + "/data/original/groundtruth/"
print("Loading " + str(n) + " images")
gt_imgs = []
for i in range (n):
    img = mpimg.imread(gt_dir + files[i])
    try:
        gt_imgs.append(img[:,:,0])
    except:
        gt_imgs.append(img)
print(files[0])

Loading 100 images
satImage_052.png


In [9]:
# Extract patches from input images
patch_size = 16 # each patch is 16*16 pixels

img_patches = [img_crop(imgs[i], patch_size, patch_size) for i in range(n)]
gt_patches = [img_crop(gt_imgs[i], patch_size, patch_size) for i in range(n)]

# Linearize list of patches
img_patches = np.asarray([img_patches[i][j] for i in range(len(img_patches)) for j in range(len(img_patches[i]))])
gt_patches =  np.asarray([gt_patches[i][j] for i in range(len(gt_patches)) for j in range(len(gt_patches[i]))])

In [11]:
# Compute features for each image patch
foreground_threshold = 0.25 # percentage of pixels > 1 required to assign a foreground label to a patch

from helpers import *

X = np.asarray([extract_features_2d(img_patches[i]) for i in range(len(img_patches))])
Y = np.asarray([value_to_class(np.mean(gt_patches[i]), foreground_threshold) for i in range(len(gt_patches))])

In [12]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=1)

## Linear Modelling Using Logistic Regression

In [17]:
logreg = LogisticRegression(C=1e5, class_weight="balanced")
logreg.fit(x_train, y_train)

LogisticRegression(C=100000.0, class_weight='balanced')

In [18]:
y_hat = logreg.predict(x_test)
f1 = f1_score(y_test, y_hat)

print(f'F1-Score: {f1}')

F1-Score: 0.4474462630644843


## Loading the Augmented Training Set for Augmented Modelling

In [22]:
image_dir = str(Path.cwd().parent) + "/data/training/images/90-split/"
# Loading the images
files = os.listdir(image_dir)
n = len(files) 
imgs = [mpimg.imread(image_dir + files[i]) for i in range(n)]

# Loading the groundtruth images
gt_dir = str(Path.cwd().parent) + "/data/training/groundtruth/90-split/"
gt_imgs = []
for i in range (n):
    img = mpimg.imread(gt_dir + files[i])
    try:
        gt_imgs.append(img[:,:,0])
    except:
        gt_imgs.append(img)

In [23]:
# Extract patches from input images
patch_size = 16 # each patch is 16*16 pixels

img_patches = [img_crop(imgs[i], patch_size, patch_size) for i in range(n)]
gt_patches = [img_crop(gt_imgs[i], patch_size, patch_size) for i in range(n)]

# Linearize list of patches
img_patches = np.asarray([img_patches[i][j] for i in range(len(img_patches)) for j in range(len(img_patches[i]))])
gt_patches =  np.asarray([gt_patches[i][j] for i in range(len(gt_patches)) for j in range(len(gt_patches[i]))])

In [25]:
foreground_threshold = 0.25
X = np.asarray([extract_features_2d(img_patches[i]) for i in range(len(img_patches))])
Y = np.asarray([value_to_class(np.mean(gt_patches[i]), foreground_threshold) for i in range(len(gt_patches))])
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=1)

## Linear Modelling Using SVM and Logistic Regression

In [26]:
logreg = LogisticRegression(C=1e5, class_weight="balanced")
logreg.fit(x_train, y_train)

LogisticRegression(C=100000.0, class_weight='balanced')

In [29]:
y_hat = logreg.predict(x_test)
f1 = f1_score(y_test, y_hat)

print(f'F1-Score: {f1}')

F1-Score: 0.41636069450494734
